# Capstone Project - The Battle of the Neighborhoods (Week 2)
### IBM/Coursera Applied Data Science Capstone

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Defining the Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a gym. Specifically, this report will be targeted to stakeholders interested in opening a Martial Arts Gym in New York, NY.

Since there are lots of gyms in NYC we will try to detect locations that are not already crowded with gyms. We are also particularly interested in areas with no Martial Arts Gyms in the vicinity. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.

We will use data science techniques to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data Definition, Sources & Gathering <a name="data"></a>

Based on definition of our problem, factors that will influence our decision are:

* number of existing gyms in the neighborhood (any type of gym)
* number of and distance to Martial Arts Gyms in the neighborhood, if any
* distance of neighborhood from city center

Regularly spaced grids of locations were used, centered around city center, to define our neighborhoods.

The following data sources will be needed to extract/generate the required information:

* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using Google Maps API reverse geocoding
* number of Gyms and their type and location in every neighborhood will be obtained using the Foursquare API
* coordinates of the NYC center will be obtained using Google Maps API geocoding

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around the NYC city center.

Let's first find the latitude & longitude of the NYC city center, using specific, well known address and Google Maps geocoding API.

In [1]:
# Perform Imports & Set API Key

In [59]:
import os
import requests
google_api_key=os.environ.get('google_geocode_apik')

In [60]:
def fetch_coordinates(api_key, address, verbose=False):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?key={api_key}&address={address}'
    response = requests.get(url).json()
    results = response['results']
    geo_data = results[0]['geometry']['location']
    lat = geo_data['lat']
    lon = geo_data['lng']
    return [lat, lon]
    
address = 'Woodside, NY'
nyc_center = fetch_coordinates(google_api_key, address)
print(f'Coordinate of {address}: {nyc_center}')

Coordinate of Woodside, NY: [40.7532952, -73.9068973]


Now let's create a grid of area candidates, equaly spaced, centered around the city center and within ~12km from Woodside, NY. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in a Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on a Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [61]:
#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('NYC center longitude={}, latitude={}'.format(nyc_center[1], nyc_center[0]))
x, y = lonlat_to_xy(nyc_center[1], nyc_center[0])
print('NYC center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('NYC center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
NYC center longitude=-73.9068973, latitude=40.7532952
NYC center UTM X=-5815056.576617461, Y=9857010.472669667
NYC center longitude=-73.90689729999957, latitude=40.7532951999989


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [62]:
nyc_center_x, nyc_center_y = lonlat_to_xy(nyc_center[1], nyc_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = nyc_center_x - 6000
x_step = 600
y_min = nyc_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(nyc_center_x, nyc_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [63]:
#!pip install folium

import folium

In [64]:
map_nyc = folium.Map(location=nyc_center, zoom_start=13)
folium.Marker(nyc_center, popup='NYC Center - Woodside').add_to(map_nyc)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_nyc) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_nyc)
    #folium.Marker([lat, lon]).add_to(map_nyc)
map_nyc

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~12km from the center of NYC. 

Let's now use Google Maps API to get approximate addresses of those locations.

In [65]:

def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, nyc_center[0], nyc_center[1])
print('Reverse geocoding check')
print('-----------------------') 
print('Address of [{}, {}] is: {}'.format(nyc_center[0], nyc_center[1], addr))


Reverse geocoding check
-----------------------
Address of [40.7532952, -73.9068973] is: Northern Blvd, Queens, NY 11377, USA


In [66]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
#     address = address.replace(', United States', '') # We don't need country part of address
    addresses.append(address)
    print(address)
#     print(' .', end='')
print(' done.')

Obtaining location addresses: 98-50 50th Ave, Corona, NY 11368, USA
101-40 Nicolls Ave, Flushing, NY 11368, USA
4102 103rd St, Corona, NY 11368, USA
37-55 104th St, Flushing, NY 11368, USA
34-02 106th St, Flushing, NY 11368, USA
32-16 107th St, Flushing, NY 11369, USA
108-46 Ditmars Blvd, Flushing, NY 11369, USA
94-1 57th Ave, Queens, NY 11373, USA
52-7 Junction Blvd, Corona, NY 11368, USA
96-11 Alstyne Ave, Flushing, NY 11368, USA
41 97th Pl, Corona, NY 11368, USA
37-64 100th St, Flushing, NY 11368, USA
101-08 35th Ave, Flushing, NY 11368, USA
102-3 Northern Blvd, East Elmhurst, NY 11369, USA
103-2 Astoria Blvd, East Elmhurst, NY 11369, USA
104-07 27th Ave, Flushing, NY 11369, USA
104 Ditmars Blvd, East Elmhurst, NY 11369, USA
Woodhaven Blvd /Wetherole St, Queens, NY 11374, USA
91- 20 57th Ave, Queens, NY 11373, USA
90-60 53rd Ave, Flushing, NY 11373, USA
93-01 49th Ave, Queens, NY 11373, USA
94-16 42nd Ave, Flushing, NY 11373, USA
95-30 Roosevelt Ave, Jackson Heights, NY 11372, USA
3

Laurel Hill Blvd, Woodside, NY 11377, USA
53-15 Queens Blvd, Woodside, NY 11377, USA
52-30 39th Dr, Flushing, NY 11377, USA
52-16 Barnett Ave, Sunnyside, NY 11104, USA
50-49 Newtown Rd, Woodside, NY 11377, USA
48-5 Broadway, Woodside, NY 11377, USA
46-12 30th Rd, Long Island City, NY 11103, USA
43-20 28th Ave, Astoria, NY 11103, USA
41-09 25th Ave, Long Island City, NY 11103, USA
23-33 37th St, Long Island City, NY 11105, USA
22-25 33rd St, Long Island City, NY 11105, USA
21-19 29th St, Long Island City, NY 11105, USA
20-10 26th St, Long Island City, NY 11105, USA
21-20 16th Ave, Queens, NY 11105, USA
52-17 Grand Ave, Maspeth, NY 11378, USA
56-01 Maspeth Ave, Maspeth, NY 11378, USA
55-64 56th St, Maspeth, NY 11378, USA
53 Queens Midtown Expy, Woodside, NY 11377, USA
Laurel Hill Blvd, Woodside, NY 11377, USA
48-58 49th St, Woodside, NY 11377, USA
45-44 50th St, Flushing, NY 11377, USA
49-07 43rd Ave, Woodside, NY 11377, USA
39-43 48th St, Long Island City, NY 11104, USA
4818 48th St, Fl

In [67]:
addresses[-50:]

['1950 35th Ave, Queens, NY 11106, USA',
 '33-38 14th St, Long Island City, NY 11106, USA',
 '11-40 31st Dr, Queens, NY 11106, USA',
 '3002 Vernon Blvd, Astoria, NY 11102, USA',
 '2-23 27th Ave, Astoria, NY 11102, USA',
 '1-09 26th Ave, Long Island City, NY 11102, USA',
 '360 Kingsland Ave, Brooklyn, NY 11222, USA',
 '5415 35th St, Long Island City, NY 11101, USA',
 '52-01 29th St, Long Island City, NY 11101, USA',
 '47-60 29th St, Long Island City, NY 11101, USA',
 '27-20 Skillman Ave, Long Island City, NY 11101, USA',
 '42 Rd/Hunter St, Queens, NY 11101, USA',
 '2506 41st Ave, Long Island City, NY 11101, USA',
 '39-01 22nd St, Long Island City, NY 11101, USA',
 '13-01 37th Ave, Long Island City, NY 11101, USA',
 '11-2 35th Ave, Astoria, NY 11106, USA',
 '33-11 9th St, Queens, NY 11106, USA',
 '150-41 Vernon Blvd, Queens, NY 11106, USA',
 '104 Astoria Blvd, Astoria, NY 11102, USA',
 '26-40 1st St, Long Island City, NY 11102, USA',
 '403 Greenpoint Ave, Brooklyn, NY 11222, USA',
 '540 

Looking good. Let's now place all this into a Pandas dataframe.

In [68]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"98-50 50th Ave, Corona, NY 11368, USA",40.741686,-73.863013,-5.816857e+06,9.851295e+06,5992.495307
1,"101-40 Nicolls Ave, Flushing, NY 11368, USA",40.745222,-73.862872,-5.816257e+06,9.851295e+06,5840.376700
2,"4102 103rd St, Corona, NY 11368, USA",40.748759,-73.862731,-5.815657e+06,9.851295e+06,5747.173218
3,"37-55 104th St, Flushing, NY 11368, USA",40.752296,-73.862591,-5.815057e+06,9.851295e+06,5715.767665
4,"34-02 106th St, Flushing, NY 11368, USA",40.755833,-73.862450,-5.814457e+06,9.851295e+06,5747.173218
5,"32-16 107th St, Flushing, NY 11369, USA",40.759370,-73.862309,-5.813857e+06,9.851295e+06,5840.376700
6,"108-46 Ditmars Blvd, Flushing, NY 11369, USA",40.762907,-73.862168,-5.813257e+06,9.851295e+06,5992.495307
7,"94-1 57th Ave, Queens, NY 11373, USA",40.736475,-73.867249,-5.817757e+06,9.851814e+06,5855.766389
8,"52-7 Junction Blvd, Corona, NY 11368, USA",40.740011,-73.867109,-5.817157e+06,9.851814e+06,5604.462508
9,"96-11 Alstyne Ave, Flushing, NY 11368, USA",40.743547,-73.866969,-5.816557e+06,9.851814e+06,5408.326913


...and let's now save/persist this data into local file.

In [69]:
df_locations.to_pickle('./locations.pkl')

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on gyms in each neighborhood.

We're interested in venues in 'Gyms / Fitness' category, and we'll make sure to detect and include all the subcategories of specific 'Martial Arts Gym' category, as we need info on Martial Arts Dojos in the neighborhood.

Foursquare credentials are defined bellow.

In [70]:
CLIENT_ID = os.environ.get('foursq_client_id') # your Foursquare ID
CLIENT_SECRET = os.environ.get('foursq_client_secret') # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [71]:
#Get list of venue categories around New York
LIMIT = 1000 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={nyc_center[0]},{nyc_center[1]}&radius={radius}&limit={LIMIT}'

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edec89c9388d7001b3f88d8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Woodside',
  'headerFullLocation': 'Woodside, Queens',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 150,
  'suggestedBounds': {'ne': {'lat': 40.76229520900001,
    'lng': -73.89503871315469},
   'sw': {'lat': 40.744295190999985, 'lng': -73.9187558868453}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5a466fc5610f0447488800e2',
       'name': 'Xi’an Famous Foods Central Kitchen',
       'location': {'address': '55-01 37th Ave',
        'lat': 40.750899,
        'lng': -73.906499,
        'labeledLatLngs': [{'l

In [72]:
#find Gym category ID
from pprint import pprint

for i in results['response']:
    print(i)

print(len(results['response']['groups']))

gymdictlst=[]
for i in results['response']['groups'][0]['items']:
#     pprint(i['venue']['categories'][0])
    if 'Gym' in i['venue']['categories'][0]['pluralName']:
        gymdictlst.append(i['venue']['categories'][0])
    
gymdictlst

suggestedFilters
headerLocation
headerFullLocation
headerLocationGranularity
totalResults
suggestedBounds
groups
1


[{'id': '4bf58dd8d48988d175941735',
  'name': 'Gym / Fitness Center',
  'pluralName': 'Gyms or Fitness Centers',
  'shortName': 'Gym / Fitness',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
   'suffix': '.png'},
  'primary': True},
 {'id': '4bf58dd8d48988d175941735',
  'name': 'Gym / Fitness Center',
  'pluralName': 'Gyms or Fitness Centers',
  'shortName': 'Gym / Fitness',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
   'suffix': '.png'},
  'primary': True},
 {'id': '4bf58dd8d48988d176941735',
  'name': 'Gym',
  'pluralName': 'Gyms',
  'shortName': 'Gym',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
   'suffix': '.png'},
  'primary': True},
 {'id': '4bf58dd8d48988d175941735',
  'name': 'Gym / Fitness Center',
  'pluralName': 'Gyms or Fitness Centers',
  'shortName': 'Gym / Fitness',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
   'suffix': '.png'},
  'primary

In [73]:
for i in results['response']['groups'][0]['items']:
    pprint(i['venue']['categories'][0]['pluralName'])

'Chinese Restaurants'
'Nightclubs'
'Juice Bars'
'Donut Shops'
'Pizza Places'
'American Restaurants'
'Health Food Stores'
'Italian Restaurants'
'Bars'
'Convenience Stores'
'Taco Places'
'German Restaurants'
'Nightclubs'
'Martial Arts Dojos'
'Butchers'
'Gyms or Fitness Centers'
'Parks'
'Arepa Restaurants'
'South American Restaurants'
'Bakeries'
'Chinese Restaurants'
'Chinese Restaurants'
'Bars'
'Himalayan Restaurants'
'Pizza Places'
'Sushi Restaurants'
'Thai Restaurants'
'Bakeries'
'Yoga Studios'
'Gyms or Fitness Centers'
'Ramen Restaurants'
'Fast Food Restaurants'
'Pubs'
'Coffee Shops'
'Bars'
'American Restaurants'
'Latin American Restaurants'
'Bars'
'Burger Joints'
'Eastern European Restaurants'
'Bakeries'
'Thai Restaurants'
'Parks'
'Middle Eastern Restaurants'
'Fast Food Restaurants'
'French Restaurants'
'Fried Chicken Joints'
'Pet Stores'
'Bagel Shops'
'Pubs'
'Thai Restaurants'
'American Restaurants'
'Coffee Shops'
'Bars'
'Sushi Restaurants'
'Rental Car Locations'
'American Restauran

In [74]:
fsgymid='4bf58dd8d48988d175941735'
martial_arts_fs_id = '4bf58dd8d48988d101941735'

In [75]:
# Category IDs on Foursquare web site (https://developer.foursquare.com/docs/resources/categories)

gym_category = fsgymid # 'Root' category for all gym-related venues

martial_arts_cat = martial_arts_fs_id

def is_gym(categories, specific_filter=None):
    gym_words = ['gym', 'Gym', 'gyms', 'Gyms']
    gym = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in gym_words:
            if r in category_name:
                gym = True
#         if 'Martial Arts' in category_name:
#             gym = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            gym = True
    return gym, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [76]:
import pickle

def get_gyms(lats, lons):
    gyms = {}
    ma_gyms = {}
    location_gyms = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any gym (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, gym_category, CLIENT_ID, CLIENT_SECRET, radius=350, limit=100)
        area_gyms = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_g, is_ma = is_gym(venue_categories, specific_filter=martial_arts_cat)
            if is_g:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                gym = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_ma, x, y)
                if venue_distance<=300:
                    area_gyms.append(gym)
                gyms[venue_id] = gym
                if is_ma:
                    ma_gyms[venue_id] = gym
        location_gyms.append(area_gyms)
        print(' .', end='')
    print(' done.')
    return gyms, ma_gyms, location_gyms

# Try to load from local file system in case we did this before
gyms = {}
ma_gyms = {}
location_gyms = []
loaded = False
try:
    with open('gyms_350.pkl', 'rb') as f:
        gyms = pickle.load(f)
    with open('ma_gyms_350.pkl', 'rb') as f:
        ma_gyms = pickle.load(f)
    with open('location_gyms_350.pkl', 'rb') as f:
        location_gyms = pickle.load(f)
    print('Gym data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    gyms, ma_gyms, location_gyms = get_gyms(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('gyms_350.pkl', 'wb') as f:
        pickle.dump(gyms, f)
    with open('ma_gyms_350.pkl', 'wb') as f:
        pickle.dump(ma_gyms, f)
    with open('location_gyms_350.pkl', 'wb') as f:
        pickle.dump(location_gyms, f)
        

Gym data loaded.


In [77]:
import numpy as np

print('Total number of gyms:', len(gyms))
print('Total number of Martial Arts Dojos:', len(ma_gyms))
print('Percentage of Martial Arts Dojos: {:.2f}%'.format(len(ma_gyms) / len(gyms) * 100))
print('Average number of gyms in neighborhood:', np.array([len(r) for r in location_gyms]).mean())

Total number of gyms: 258
Total number of Martial Arts Dojos: 33
Percentage of Martial Arts Dojos: 12.79%
Average number of gyms in neighborhood: 1.282967032967033


In [78]:
print('List of all gyms')
print('-----------------------')
for r in list(gyms.values())[:10]:
    print(r)
print('...')
print('Total:', len(gyms))

List of all gyms
-----------------------
('5032bb1ae4b0a81e08dcea88', 'Turbo Jam', 40.743348800072575, -73.86022376650897, 'Corona, NY 11368, United States', 305, False, -5816563.824867777, 9850943.669013994)
('5032b948e4b0feaf1a5815a0', 'Team Beach Body', 40.743255615234375, -73.8601303100586, 'Corona, NY 11368, United States', 318, False, -5816579.255822855, 9850931.140623344)
('4f2854eee4b055df286014a5', 'Xtreme Fitness', 40.74964386760678, -73.86522963352094, 'Jackson Heights, NY 11372, United States', 164, False, -5815516.296461998, 9851621.291540632)
('5326e732498eed3bc8151eae', 'Xtreme Fitness Gym', 40.74965878060291, -73.86521458625793, '100-12 Roosevelt Ave, Corona, NY 11368, United States', 163, False, -5815513.71020383, 9851619.428230746)
('4e30c3c381dc65de51a337b5', 'United TaeKwonDo Center Corona School', 40.75028264745698, -73.8656242914668, '40-10 National St, Corona, NY 11368, United States', 97, True, -5815409.56168368, 9851675.43126088)
('4d408328915a37040b38167f', 'L

In [79]:
print('List of Martial Arts Dojos')
print('---------------------------')
for r in list(ma_gyms.values())[:10]:
    print(r)
print('...')
print('Total:', len(ma_gyms))

List of Martial Arts Dojos
---------------------------
('4e30c3c381dc65de51a337b5', 'United TaeKwonDo Center Corona School', 40.75028264745698, -73.8656242914668, '40-10 National St, Corona, NY 11368, United States', 97, True, -5815409.56168368, 9851675.43126088)
('55f4589c498e0794ff1956f1', 'Tristate Martial Arts Institute', 40.73549848213117, -73.87281152101782, '85-04 Queens Blvd, Elmhurst, NY 11373, United States', 246, True, -5817943.655441224, 9852526.696271645)
('4e75002d8998ed82a4546a74', 'DoMA TaeKwonDo JH', 40.74891756570473, -73.87146205128845, 'New York, United States', 347, True, -5815663.591150319, 9852420.969462162)
('4e51240ce4cdc76dc9c85903', 'DoM\tA Tkd', 40.749122619628906, -73.87116241455078, 'New York, United States', 37, True, -5815627.674681875, 9852383.388243847)
('5dfa55ccf3df6f0008f5ff42', 'United Xtreme Martial Arts', 40.7380581, -73.8816611, '81-15 Queens Blvd, Elmhurst, NY 11373, United States', 289, True, -5817543.790611963, 9853680.920569172)
('4c802635d4

In [80]:
print('Gyms around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_gyms[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Gyms around location {}: {}'.format(i+1, names))

Gyms around location
---------------------------
Gyms around location 101: Urban Jungle
Gyms around location 102: 
Gyms around location 103: 
Gyms around location 104: New Self Image, Blink Fitness, S.Y. Kims Tae Kwan Doe Center
Gyms around location 105: 
Gyms around location 106: 
Gyms around location 107: 
Gyms around location 108: 
Gyms around location 109: 
Gyms around location 110: Shorehaven Gym


Let's now see all the collected gyms in our area of interest on map, and let's also show Martial Arts Dojos in a  different color.

In [81]:
map_nyc = folium.Map(location=nyc_center, zoom_start=13)
folium.Marker(nyc_center, popup='NYC Center').add_to(map_nyc)
for res in gyms.values():
    lat = res[2]; lon = res[3]
    is_ma = res[6]
    color = 'red' if is_ma else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_nyc)
map_nyc

So now we have all the gyms in area within few kilometers from the NYC center, and we know which ones are Martial Arts Dojos. We also know which gyms exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Martial Arts Dojo!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of NYC that have low gym density, particularly those with low number of Martial Arts dojos. We will limit our analysis to area ~5km around city center.

In first step we have collected the required **data: location and type (category) of every gym within 6km from NYC center** (Woodside). We have also **identified Martial Arts Dojos** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**gym density**' across different areas of NYC - we will use **heatmaps** to identify a few promising areas close to center with low number of gyms in general (*and* no Martial Arts Dojos in the vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two gyms in radius of 200 meters**, and we want locations **without Martial Arts Dojos in radius of 400 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of gyms in every area candidate**:

In [82]:
location_gyms_count = [len(res) for res in location_gyms]

df_locations['Gyms in area'] = location_gyms_count

print('Average number of gyms in every area with radius=200m:', np.array(location_gyms_count).mean())

df_locations.head(10)

Average number of gyms in every area with radius=200m: 1.282967032967033


,Address,Latitude,Longitude,X,Y,Distance from center,Gyms in area
0,"98-50 50th Ave, Corona, NY 11368, USA",40.741686,-73.863013,-5.816857e+06,9.851295e+06,5992.495307,2
1,"101-40 Nicolls Ave, Flushing, NY 11368, USA",40.745222,-73.862872,-5.816257e+06,9.851295e+06,5840.376700,0
2,"4102 103rd St, Corona, NY 11368, USA",40.748759,-73.862731,-5.815657e+06,9.851295e+06,5747.173218,3
3,"37-55 104th St, Flushing, NY 11368, USA",40.752296,-73.862591,-5.815057e+06,9.851295e+06,5715.767665,0
4,"34-02 106th St, Flushing, NY 11368, USA",40.755833,-73.862450,-5.814457e+06,9.851295e+06,5747.173218,3
5,"32-16 107th St, Flushing, NY 11369, USA",40.759370,-73.862309,-5.813857e+06,9.851295e+06,5840.376700,4
6,"108-46 Ditmars Blvd, Flushing, NY 11369, USA",40.762907,-73.862168,-5.813257e+06,9.851295e+06,5992.495307,2
7,"94-1 57th Ave, Queens, NY 11373, USA",40.736475,-73.867249,-5.817757e+06,9.851814e+06,5855.766389,0
8,"52-7 Junction Blvd, Corona, NY 11368, USA",40.740011,-73.867109,-5.817157e+06,9.851814e+06,5604.462508,0
9,"96-11 Alstyne Ave, Flushing, NY 11368, USA",40.743547,-73.866969,-5.816557e+06,9.851814e+06,5408.326913,0


OK, now let's calculate the **distance to nearest Martial Arts Dojo from every area candidate center** (not only those within 200m - we want distance to closest one, regardless of how distant it is).

In [83]:
distances_to_ma_gym = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in ma_gyms.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_ma_gym.append(min_distance)

df_locations['Distance to Martial Arts Gym'] = distances_to_ma_gym

In [84]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Gyms in area,Distance to Martial Arts Gym
0,"98-50 50th Ave, Corona, NY 11368, USA",40.741686,-73.863013,-5.816857e+06,9.851295e+06,5992.495307,2,1496.263580
1,"101-40 Nicolls Ave, Flushing, NY 11368, USA",40.745222,-73.862872,-5.816257e+06,9.851295e+06,5840.376700,0,928.647824
2,"4102 103rd St, Corona, NY 11368, USA",40.748759,-73.862731,-5.815657e+06,9.851295e+06,5747.173218,3,453.837922
3,"37-55 104th St, Flushing, NY 11368, USA",40.752296,-73.862591,-5.815057e+06,9.851295e+06,5715.767665,0,519.182953
4,"34-02 106th St, Flushing, NY 11368, USA",40.755833,-73.862450,-5.814457e+06,9.851295e+06,5747.173218,3,1026.222694
5,"32-16 107th St, Flushing, NY 11369, USA",40.759370,-73.862309,-5.813857e+06,9.851295e+06,5840.376700,4,1598.973139
6,"108-46 Ditmars Blvd, Flushing, NY 11369, USA",40.762907,-73.862168,-5.813257e+06,9.851295e+06,5992.495307,2,2186.389073
7,"94-1 57th Ave, Queens, NY 11373, USA",40.736475,-73.867249,-5.817757e+06,9.851814e+06,5855.766389,0,736.531117
8,"52-7 Junction Blvd, Corona, NY 11368, USA",40.740011,-73.867109,-5.817157e+06,9.851814e+06,5604.462508,0,1061.589693
9,"96-11 Alstyne Ave, Flushing, NY 11368, USA",40.743547,-73.866969,-5.816557e+06,9.851814e+06,5408.326913,0,1079.558389


In [85]:
print('Average distance to closest Martial Arts Gym from each area center:', df_locations['Distance to Martial Arts Gym'].mean())

Average distance to closest Martial Arts Gym from each area center: 1361.3854441874048


So, **on average Martial Arts Gyms can be found within ~1400m** from every area center candidate. That's fairly far. It seems like there is plenty of space and potential for starting a Martial Arts Gym.

To visualize the spatial distribution of gyms and Martial Arts dojos around NYC, let's create a heatmap showing **the density of gyms**. Also, let's show **borders of NYC boroughs** on our map and a few circles indicating distance of 1km, 2km and 3km from NYC Center.

In [86]:
nyc_boroughs_url = 'https://github.com/musicalchemist/Coursera_Capstone/raw/master/Borough%20Boundaries.geojson'

nyc_boroughs = requests.get(nyc_boroughs_url).json()

def boroughs_style(feature):
    return { 'color': 'blue', 'fill': False }

nyc_boroughs

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'boro_code': '2',
    'boro_name': 'Bronx',
    'shape_area': '1186612478.27',
    'shape_leng': '462958.187578'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-73.89680883223774, 40.79580844515979],
       [-73.89693872998792, 40.79563587285357],
       [-73.89723603843939, 40.79572003753707],
       [-73.89796839783742, 40.795644839161994],
       [-73.89857332665558, 40.7960691402596],
       [-73.89895261832527, 40.796227852579634],
       [-73.89919434249981, 40.79650245601822],
       [-73.89852052071471, 40.796936194189776],
       [-73.89788253240185, 40.79711653214705],
       [-73.89713149795642, 40.79679807772831],
       [-73.89678526341234, 40.796329166487105],
       [-73.89680883223774, 40.79580844515979]]],
     [[[-73.88885148496334, 40.798706328958765],
       [-73.88860021869873, 40.798650985918705],
       [-73.8885856250733, 40.798706072297094],
       [-73.888213488

In [87]:
gym_latlons = [[res[2], res[3]] for res in gyms.values()]

ma_latlons = [[res[2], res[3]] for res in ma_gyms.values()]

## NYC Gym Heatmap

In [93]:
from folium import plugins
from folium.plugins import HeatMap

map_nyc = folium.Map(location=nyc_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_nyc) #cartodbpositron cartodbdark_matter
HeatMap(gym_latlons).add_to(map_nyc)
folium.Marker(nyc_center).add_to(map_nyc)
folium.Circle(nyc_center, radius=1000, fill=False, color='white').add_to(map_nyc)
folium.Circle(nyc_center, radius=2000, fill=False, color='white').add_to(map_nyc)
folium.Circle(nyc_center, radius=3000, fill=False, color='white').add_to(map_nyc)
folium.GeoJson(nyc_boroughs, style_function=boroughs_style, name='geojson').add_to(map_nyc)
map_nyc.save("hm1.html")

Looks like a few pockets of low gym density closest to city center can be found.

Let's create another heatmap map showing **the density of Martial Arts gyms** only.

## NYC Martial Arts Gym Heatmap

In [94]:
map_nyc = folium.Map(location=nyc_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_nyc) #cartodbpositron cartodbdark_matter
HeatMap(ma_latlons).add_to(map_nyc)
folium.Marker(nyc_center).add_to(map_nyc)
folium.Circle(nyc_center, radius=1000, fill=False, color='white').add_to(map_nyc)
folium.Circle(nyc_center, radius=2000, fill=False, color='white').add_to(map_nyc)
folium.Circle(nyc_center, radius=3000, fill=False, color='white').add_to(map_nyc)
folium.GeoJson(nyc_boroughs, style_function=boroughs_style, name='geojson').add_to(map_nyc)
map_nyc.save("hm2.html")

This map is shows that there are several pockets of **low density Martial Arts Gyms positioned north, east, and south from city center**.

Based on this we will now focus our analysis on these areas.

Let's define new, more narrow region of interest, which will include these low-martial arts-count parts closest to NYC.

In [95]:
roi_x_min = nyc_center_x - 2000
roi_y_max = nyc_center_y + 1000
roi_width = 5000
roi_height = 5000
roi_center_x = roi_x_min + 2500
roi_center_y = roi_y_max - 2500
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

map_nyc = folium.Map(location=roi_center, zoom_start=14)
HeatMap(gym_latlons).add_to(map_nyc)
folium.Marker(nyc_center).add_to(map_nyc)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_nyc)
folium.GeoJson(nyc_boroughs, style_function=boroughs_style, name='geojson').add_to(map_nyc)
map_nyc.save("hm3.html")

Let's also create new, more dense grid of location candidates restricted to our new region of interest (let's make our location candidates 100m appart).

In [96]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 100
y_step = 100 * k 
roi_y_min = roi_center_y - 2500

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 2501):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

2261 candidate neighborhood centers generated.


OK. Now let's calculate two most important things for each location candidate: **number of gyms in the vicinity** (we'll use radius of **250 meters**) and **distance to closest Martial Arts gym**.

In [97]:
def count_gyms_nearby(x, y, gyms, radius=250):    
    count = 0
    for res in gyms.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_gym(x, y, gyms):
    d_min = 100000
    for res in gyms.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_gym_counts = []
roi_ma_gym_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_gyms_nearby(x, y, gyms, radius=250)
    roi_gym_counts.append(count)
    distance = find_nearest_gym(x, y, ma_gyms)
    roi_ma_gym_distances.append(distance)
print('done.')


Generating data on location candidates... done.


In [98]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Gyms nearby':roi_gym_counts,
                                 'Distance to MA Gym':roi_ma_gym_distances})

df_roi_locations['Gyms nearby'].max()
df_roi_locations

,Latitude,Longitude,X,Y,Gyms nearby,Distance to MA Gym
0,40.755253,-73.875786,-5.814607e+06,9.853010e+06,0,975.524241
1,40.755842,-73.875763,-5.814507e+06,9.853010e+06,0,976.350579
2,40.752026,-73.876585,-5.815157e+06,9.853097e+06,0,845.093320
3,40.752615,-73.876562,-5.815057e+06,9.853097e+06,0,908.617425
4,40.753205,-73.876539,-5.814957e+06,9.853097e+06,0,970.666162
...,...,...,...,...,...,...
2256,40.756992,-73.914021,-5.814457e+06,9.857947e+06,0,320.039737
2257,40.757582,-73.913999,-5.814357e+06,9.857947e+06,0,295.121772
2258,40.758171,-73.913977,-5.814257e+06,9.857947e+06,0,299.139474
2259,40.758761,-73.913954,-5.814157e+06,9.857947e+06,0,334.472704


We'll now **filter** those locations: we're interested only in **locations with no more than two gyms in radius of 250 meters**, and **no Martial Arts Gyms in radius of 400 meters**.

In [113]:
good_res_count = np.array((df_roi_locations['Gyms nearby']<=2))
print('Locations with no more than two gyms nearby:', good_res_count.sum())

good_ita_distance = np.array(df_roi_locations['Distance to MA Gym']>=400)
print('Locations with no MA Gyms within 400m:', good_ita_distance.sum())

good_locations = np.logical_and(good_res_count, good_ita_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


Locations with no more than two gyms nearby: 2244
Locations with no MA Gyms within 400m: 2136
Locations with both conditions met: 2126


Let's see how this looks on a map.

In [100]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_nyc = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_nyc)
HeatMap(gym_latlons).add_to(map_nyc)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.6).add_to(map_nyc)
folium.Marker(nyc_center).add_to(map_nyc)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_nyc) 
folium.GeoJson(nyc_boroughs, style_function=boroughs_style, name='geojson').add_to(map_nyc)
map_nyc.save('hm4.html')

Good, we now have locations of potential candidates for a new Martial Arts Gym, at least based on nearby competition.

Let's now show those good locations in a form of heatmap:

In [101]:
map_nyc = folium.Map(location=roi_center, zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_nyc)
folium.Marker(nyc_center).add_to(map_nyc)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_nyc)
folium.GeoJson(nyc_boroughs, style_function=boroughs_style, name='geojson').add_to(map_nyc)
map_nyc.save('hm5.html')

Looking good. What we have now is a clear indication of zones with low number of gyms in vicinity, and *no* Martial Arts gyms at all nearby.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

## KMeans Cluster Analysis

In [103]:
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_nyc = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_nyc)
HeatMap(gym_latlons).add_to(map_nyc)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_nyc)
folium.Marker(nyc_center).add_to(map_nyc)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_nyc) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_nyc)
folium.GeoJson(nyc_boroughs, style_function=boroughs_style, name='geojson').add_to(map_nyc)
map_nyc.save('hm6.html')

Not bad - our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [114]:
map_nyc = folium.Map(location=roi_center, zoom_start=14)
folium.Marker(nyc_center).add_to(map_nyc)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_nyc)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_nyc)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_nyc) 
folium.GeoJson(nyc_boroughs, style_function=boroughs_style, name='geojson').add_to(map_nyc)
map_nyc.save('hm7.html')

## Reverse Geocoding to find Candidate Addresses

Finaly, let's **reverse geocode those candidate area centers to get the addresses** which can be presented to stakeholders.

In [105]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(google_api_key, lat, lon)
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, nyc_center_x, nyc_center_y)
    print('{}{} => {:.1f}km from NYC Center'.format(addr, ' '*(50-len(addr)), d/1000))
    

Addresses of centers of areas recommended for further analysis

28-45 47th St, Queens, NY 11103, USA               => 1.4km from NYC Center
87-7 35th Ave, Jackson Heights, NY 11372, USA      => 3.5km from NYC Center
39-36 63rd St, Woodside, NY 11377, USA             => 1.5km from NYC Center
23-58 83rd St, Flushing, NY 11370, USA             => 3.4km from NYC Center
75-25 31st Ave, East Elmhurst, NY 11370, USA       => 2.3km from NYC Center
41-31 71st St, Flushing, NY 11377, USA             => 2.3km from NYC Center
62-5 30th Ave, Woodside, NY 11377, USA             => 1.2km from NYC Center
22-37 74th St, Flushing, NY 11370, USA             => 2.8km from NYC Center
50-33 Broadway, Woodside, NY 11377, USA            => 0.5km from NYC Center
40-15 81st St, Flushing, NY 11373, USA             => 3.1km from NYC Center
30-50 88th St, Flushing, NY 11369, USA             => 3.6km from NYC Center
23-25 49th St, Long Island City, NY 11105, USA     => 2.2km from NYC Center
34-40 65th St, Flushing,

This concludes our analysis. We have created 15 addresses representing centers of zones containing locations with low number of gyms and no Martial Arts Gyms nearby, all zones being fairly close to city center (all less than 4km from Woodside, NYC (the city center).

In [106]:
map_nyc = folium.Map(location=roi_center, zoom_start=14)
folium.Circle(nyc_center, radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_nyc)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]], popup=addr).add_to(map_nyc) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_nyc)
map_nyc

## Results and Discussion <a name="results"></a>

Our analysis shows that although there are a decent number of gyms in NYC. There are pockets of low martial arts gym density fairly close to city center. The highest concentration of gyms was detected west of the city center, so we focused our attention to areas north, east and south.

After directing our attention to this more narrow area of interest (covering approx. 5x5km south-east from the city center) we first created a dense grid of location candidates (spaced 100m appart); those locations were then filtered so that those with more than two gyms in radius of 250m and those with a martial arts gym closer than 400m were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 15 zones containing largest number of potential new gym locations based on number of and distance to existing venues - both gyms in general and Martial Arts Gyms particularly. This, of course, does not imply that those zones are actually optimal locations for a new gym. The purpose of this analysis was to only provide info on areas close to the NYC center but not crowded with existing gyms. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

The purpose of this project was to identify NYC areas close to the city center with a low number of gyms, and in particular, a low number of Martial Arts gyms, in order to aid stakeholders in narrowing down the search for optimal location for a new Martial Arts Dojo in NYC. By calculating gym density distribution from Foursquare data we have first identified general boroughs that justify further analysis (south, north, and east of the city center in Woodside, NYC), and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby gyms. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

The final decision on optimal gym location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.